# Data Mining HW2 (Kaggle)

---
## 1. Data Preprocessing

https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework/overview

In [32]:
import pandas as pd
import json

# # file = open("tweets_DM.json", "r", encoding="utf-8")
# # data = []

# # for line in file.readlines():
# #     dic = json.loads(line)
# #     data.append(dic)

# # data = pd.DataFrame(data)
# # print(data.shape)

# ## save to pickle file
# data.to_pickle("data.pkl")


data = pd.read_pickle("data.pkl")
identification = pd.read_csv("data_identification.csv")

# Extract the "tweet_id" of each data and add a new column to record it.
data["tweet_id"] = data["_source"].apply(lambda x : x["tweet"]["tweet_id"])
# Add a column to record the data type (train or test) according to "identification"
data = data.merge(identification, on = "tweet_id", how = "right")



In [33]:
# Obseving that the value in data[_index] are all the same, I decide to delete this column.
from collections import Counter
count_data = Counter(data["_index"])
print(len(data))
print(count_data["hashtag_tweets"])

data.drop(["_index"], axis = 1, inplace=True)

# I think the crawldate might not affect the emotion
data.drop(["_crawldate"], axis = 1, inplace=True)

# Similarly for column "_type"
count_data = Counter(data["_type"])
print(count_data["tweets"])

data.drop(["_type"], axis = 1, inplace=True)


1867535
1867535
1867535


In [34]:
# Then I extract the "hashtags" and "text" content in "_source" to form two new columns in data.
# After doing this, we can eliminate the "source" column.

data["hashtags"] = data["_source"].apply(lambda x : x["tweet"]["hashtags"])
data["text"] = data["_source"].apply(lambda x : x["tweet"]["text"])

data.drop(["_source"], axis = 1, inplace=True)

In [35]:
# We then add the "emotion" tags to a new column named "emotion" according to "emotion.csv".
emotion = pd.read_csv("emotion.csv")
data = data.merge(emotion, on = "tweet_id", how = "outer")

In [36]:
# Combine the hashtags to the text, then we only need to focus on the "text" data.
data.text.append(data.hashtags)
data.drop(["hashtags"],axis=1, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_16644\2630931373.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.text.append(data.hashtags)


In [37]:
import re
import string

# # change all the big latter to small latter
data.text = data.text.str.lower()
print(data.text[0])

# # delete the hashtag, mention, punctuation and numbers
data.text = data.text.apply(lambda x: re.sub(r'@', '', x))
data.text = data.text.apply(lambda x: re.sub(r'#', '', x))
data.text = data.text.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
data.text = data.text.apply(lambda x: re.sub(r'\d+', '', x))
data.text = data.text.apply(lambda x: re.sub(r'lh', '', x))
print(data.text[0])


@habbo i've seen two separate colours of the elegant furni on your homepage? <lh>
habbo ive seen two separate colours of the elegant furni on your homepage 


In [1]:
import pandas as pd

# ### training data
# train_df = data[data["emotion"].notnull()]
# train_df.head()
# train_df.to_pickle("train_df.pkl")
# print(len(train_df))

# ### testing data
# Submission = data[data["emotion"].isna()]
# Submission.head()
# Submission.to_pickle("Submission.pkl")
# print(len(Submission))

train_df = pd.read_pickle("train_df.pkl")
Submission = pd.read_pickle("Submission.pkl")



In [2]:
# shuffle dataset
train_df = train_df.sample(frac=1)
Submission = Submission.sample(frac=1)

In [3]:
print("Shape of Training df: ", train_df.shape)
print("Shape of Testing df: ", Submission.shape)

Shape of Training df:  (1455563, 5)
Shape of Testing df:  (411972, 5)


In [4]:
# classes = set(train_df["emotion"])

# num = []
# for i, emo in zip(range(8), classes):
#     num.append(len(train_df[train_df["emotion"] == emo]))

# print(num)
# print(sum(num) == len(train_df))

# Try to apply LSTM method

In [3]:
# LSTM
import nltk
import string

#train_df['text_tokenized'] = train_df['text'].apply(lambda x: nltk.word_tokenize(x))
#Submission['text_tokenized'] = Submission['text'].apply(lambda x: nltk.word_tokenize(x))


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

NUM_WORDS = 10000

tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token='<UNK>')
tokenizer.fit_on_texts(train_df.text)



In [8]:
# Obesrving that most article are with less than 40 words.

# import matplotlib.pyplot as plt
# lengths = [len(t.split(' ')) for t in train_df["text"]]

# print(max(lengths))
# plt.hist(lengths, bins=len(set(lengths)))
# plt.show()

In [5]:
# padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=50, padding='post')
    return padded_sequences


In [8]:
classes = set(train_df["emotion"])
print(classes)

{'sadness', 'trust', 'joy', 'fear', 'surprise', 'disgust', 'anger', 'anticipation'}


# LSTM with KFold

In [12]:
import tensorflow as tf
# from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras import regularizers


LSTM_model = Sequential()
LSTM_model.add(Embedding(NUM_WORDS, 30, input_length=50))
LSTM_model.add(Bidirectional(LSTM(15)))
LSTM_model.add(Dense(8, activation='softmax'))

LSTM_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

LSTM_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 30)            15000     
                                                                 
 bidirectional (Bidirectiona  (None, 30)               5520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 8)                 248       
                                                                 
Total params: 20,768
Trainable params: 20,768
Non-trainable params: 0
_________________________________________________________________


In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

classes = set(train_df["emotion"])

classes_to_index = dict((c, i) for i, c in enumerate(classes))
index_to_classes = dict((i, c) for c, i in classes_to_index.items())

print(classes_to_index)
print(index_to_classes)

names_to_ids = lambda labels: pd.Series([classes_to_index.get(x) for x in labels])
ids_to_names = lambda labels: pd.Series([index_to_classes.get(x) for x in labels])

X = get_sequences(tokenizer, train_df['text'])
y = names_to_ids(train_df["emotion"])

KFOLD = 2

skf = RepeatedStratifiedKFold(n_splits=KFOLD, n_repeats=5)
skf.get_n_splits(X, y)
print(skf)

i = 1
for train_index, test_index in skf.split(X, y):
    print(f"This is the {i} round.")
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    h = LSTM_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=10000,
    #class_weight=CLASS_WEIGHT,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)
    ]
    )
    i = i + 1





{'sadness': 0, 'fear': 1, 'anticipation': 2, 'surprise': 3, 'joy': 4, 'disgust': 5, 'anger': 6, 'trust': 7}
{0: 'sadness', 1: 'fear', 2: 'anticipation', 3: 'surprise', 4: 'joy', 5: 'disgust', 6: 'anger', 7: 'trust'}


NameError: name 'tokenizer' is not defined

In [ ]:
my_LSTM_Submission2 = pd.DataFrame()

import numpy as np

X_LSTM_test = get_sequences(tokenizer, Submission['text'])
y_LSTM_pred = LSTM_model.predict(X_LSTM_test)

# ids_to_names = lambda labels: pd.Series([index_to_classes.get(x) for x in labels])

my_LSTM_Submission2["id"] = Submission["tweet_id"]
my_LSTM_Submission2 = my_LSTM_Submission2.reset_index(drop =True)

for i in range(8):
    my_LSTM_Submission2[f"{index_to_classes.get(i)}"] = y_LSTM_pred[:,i]
my_LSTM_Submission2.to_csv("my_LSTM_Submission2.csv", index=False)  

12875/12875 [==============================] - 197s 15ms/step


# LSTM without Kfold (add ensemble)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X = get_sequences(tokenizer, train_df['text'])
y = train_df['emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)


## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1310006, 50)
y_train.shape:  (1310006,)
X_test.shape:  (145557, 50)
y_test.shape:  (145557,)


In [9]:
# import pandas as np

classes_to_index = dict((c, i) for i, c in enumerate(classes))
index_to_classes = dict((i, c) for c, i in classes_to_index.items())

print(classes_to_index)
print(index_to_classes)

names_to_ids = lambda labels: pd.Series([classes_to_index.get(x) for x in labels])
ids_to_names = lambda labels: pd.Series([index_to_classes.get(x) for x in labels])
y_train_lstm = names_to_ids(y_train)
y_test_lstm = names_to_ids(y_test)

print(y_train_lstm[0:5])
print(y_test_lstm[0:5])
print(y_train_lstm.shape)
print(y_test_lstm.shape)

{'sadness': 0, 'trust': 1, 'joy': 2, 'fear': 3, 'surprise': 4, 'disgust': 5, 'anger': 6, 'anticipation': 7}
{0: 'sadness', 1: 'trust', 2: 'joy', 3: 'fear', 4: 'surprise', 5: 'disgust', 6: 'anger', 7: 'anticipation'}
0    2
1    7
2    1
3    0
4    5
dtype: int64
0    1
1    5
2    3
3    2
4    0
dtype: int64
(1310006,)
(145557,)


In [10]:
import tensorflow as tf
# from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras import regularizers
from keras.layers import SpatialDropout1D


LSTM_model = Sequential()
LSTM_model.add(Embedding(NUM_WORDS, 32, input_length=X_train.shape[1]))
LSTM_model.add(SpatialDropout1D(0.2))
LSTM_model.add((LSTM(16,dropout=0.2, recurrent_dropout=0.2)))
LSTM_model.add(Dense(8, activation='softmax'))

LSTM_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

LSTM_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 32)            320000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 32)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 8)                 136       
                                                                 
Total params: 323,272
Trainable params: 323,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# import numpy as np
# import math

# classes_cnt = [np.where(y_train_lstm == c)[0].shape[0] for c in range(len(classes))]
# max_class_size = np.max(classes_cnt)
# CLASS_WEIGHT = [max_class_size/size for size in classes_cnt]
# CLASS_WEIGHT = [math.sqrt(CLASS_WEIGHT[i]) for i in range(len(CLASS_WEIGHT))]
# CLASS_WEIGHT = dict(zip(set(y_train_lstm), CLASS_WEIGHT))

In [16]:
# for i in range(5):

h = LSTM_model.fit(
    X_train, y_train_lstm,
    validation_data=(X_test, y_test_lstm),
    epochs=5,
    batch_size=10000,
    # class_weight=CLASS_WEIGHT,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', patience=2)
    ]
)


Epoch 1/5
132/132 [==============================] - 124s 939ms/step - loss: 1.2572 - accuracy: 0.5444 - val_loss: 1.2754 - val_accuracy: 0.5379
Epoch 2/5
 49/132 [==========>...................] - ETA: 1:10 - loss: 1.2585 - accuracy: 0.5432

KeyboardInterrupt: 

In [35]:
AUTO = tf.data.experimental.AUTOTUNE
MODEL = 'albert-base-v2'
from transformers import AutoTokenizer

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        #return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )

max_len =  50

tokenizer = AutoTokenizer.from_pretrained(MODEL)
X_train_t = regular_encode(list(train_df["text"]), tokenizer, maxlen=max_len)
X_test_t = regular_encode(X_test, tokenizer, maxlen=max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\User\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
my_LSTM_Submission = pd.DataFrame()

import numpy as np

X_LSTM_test = get_sequences(tokenizer, Submission['text'])
y_LSTM_pred = np.argmax(LSTM_model.predict(X_LSTM_test), axis=1)

ids_to_names = lambda labels: pd.Series([index_to_classes.get(x) for x in labels])

y_LSTM_result = ids_to_names(y_LSTM_pred)
my_LSTM_Submission["id"] = Submission['tweet_id']
my_LSTM_Submission = my_LSTM_Submission.reset_index(drop =True)
my_LSTM_Submission["emotion"] = y_LSTM_result
my_LSTM_Submission.to_csv("my_LSTM_Submission.csv", index=False)

12875/12875 [==============================] - 589s 46ms/step
